In [1]:
from src.literature.common.session import Session
session = Session(for_nlp=True)

import pyspark.sql.functions as f

:: loading settings :: url = jar:file:/Users/who/.sdkman/candidates/spark/3.3.4/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/who/.ivy2/cache
The jars for the packages stored in: /Users/who/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-11bb4824-653c-49d1-b220-01b3fc514b27;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.0.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.20.1 in central
	found com.google.guava#guava;31.1-jre in central
	found com.google.guava#failureacc

25/04/29 15:00:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/04/29 15:00:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Extracing entities from Open Targets indices, then normalising the entities:

In [2]:
from src.literature.datasource.open_targets.disease import OpenTargetsDisease
from src.literature.datasource.open_targets.target import OpenTargetsTarget
from src.literature.datasource.open_targets.drug import OpenTargetsDrug

disease_entities = OpenTargetsDisease.from_index(session, "assets/disease")
disease_normalised = disease_entities.normalise_entities()

target_entities = OpenTargetsTarget.from_index(session, "assets/target")
target_normalised = target_entities.normalise_entities()

drug_entities = OpenTargetsDrug.from_index(session, "assets/drug_molecule")
drug_normalised = drug_entities.normalise_entities()

In [3]:
disease_entities.df.show()

+----------+--------------------+-----------+---------------+
|  entityId|         entityLabel|entityScore|nlpPipelineType|
+----------+--------------------+-----------+---------------+
|GO_0000226|microtubule cytos...|      0.996|           term|
|GO_0002524|    hypersensitivity|        1.0|           term|
|GO_0007165|  signalling cascade|      0.998|           term|
|GO_0007249|I-kappaB kinase/N...|      0.999|           term|
|GO_0009150|purine ribonucleo...|        1.0|           term|
|GO_1904057|down-regulation o...|      0.999|           term|
|HP_0000256|         Big cranium|      0.997|           term|
|HP_0000337| Bitemporal widening|      0.999|           term|
|HP_0000431|  Broad nasal bridge|      0.999|           term|
|HP_0000775|Diaphragmatic defect|      0.999|           term|
|HP_0000876|Light or infreque...|      0.999|           term|
|HP_0000972|Palmoplantar kera...|      0.999|           term|
|HP_0001114|         Xanthelasma|        1.0|           term|
|HP_0001

In [4]:
disease_normalised.df.show()

+----------+--------------------+-----------+---------------+---------------------+
|  entityId|         entityLabel|entityScore|nlpPipelineType|entityLabelNormalised|
+----------+--------------------+-----------+---------------+---------------------+
|GO_0000226|microtubule cytos...|      0.996|           term| biogenesicytoskel...|
|GO_0002524|    hypersensitivity|        1.0|           term|          hypersensit|
|GO_0007165|  signalling cascade|      0.998|           term|         cascadsignal|
|GO_0007249|I-kappaB kinase/N...|      0.999|           term| ikappabkinasnfsig...|
|GO_0009150|purine ribonucleo...|        1.0|           term| metabolprocesspur...|
|GO_1904057|down-regulation o...|      0.999|           term| painperceptreguls...|
|HP_0000256|         Big cranium|      0.997|           term|           bigcranium|
|HP_0000337| Bitemporal widening|      0.999|           term|        bitemporwiden|
|HP_0000431|  Broad nasal bridge|      0.999|           term|      bridgbroa

First steps of processing publications:
- process fulltexts using lookup table
- merge abstracts will fulltexts to get all epmc publications
- (should be following up deduplicating publications by getting the most recent version of each publication, demonstrated using a different dataset below)

In [5]:
# curl http://ftp.ebi.ac.uk/pub/databases/pmc/DOI/PMID_PMCID_DOI.csv.gz > PMID_PMCID_DOI.csv.gz

from src.literature.datasource.epmc.publication_id_lut import PublicationIdLUT

pub_id_lut = PublicationIdLUT.from_csv(session, "assets/PMID_PMCID_DOI.csv.gz")
pub_id_lut.count()

9684951

In [6]:
from src.literature.datasource.epmc.publication import EPMCPublication

processed_fulltexts = (
    EPMCPublication.from_source(session, "example_both_abstract_and_fulltext/fulltext/*")
    .process_fulltexts_using_lut(pub_id_lut)
)
all_epmc_publications = (
    EPMCPublication.from_source(session, "example_both_abstract_and_fulltext/abstract/*")
    .merge_abstracts_with_fulltexts(processed_fulltexts)
)
all_epmc_publications.df.count()

51671

Next steps of processing publications:
- (should be preceded by merging abstracts with fulltexts, but not done here as dataset consists of only abstracts)
- get most recent version of each publication
- extract matches
- extract entities
- normalise entitities
- (will be followed by grounding of entities to entityIds, then disambiguation)

In [7]:
normalised_entities = (
    EPMCPublication.from_source(session, "example_timestamps/abstract/*")
    .get_most_recent_publications()
    .extract_matches()
    .extract_entities()
    .normalise_entities()
)
print(normalised_entities.df.count())

174970


In [8]:
normalised_entities.df.show()

+---------------------+----------+--------------------+---------------+---------------------+
|entityLabelFromSource|entityType|         entityLabel|nlpPipelineType|entityLabelNormalised|
+---------------------+----------+--------------------+---------------+---------------------+
|                  Bax|        GP|                 Bax|           term|                  bax|
|           Homeopathy|        DS|          Homeopathy|           term|           homeopathi|
|          phytic acid|        CD|         phytic acid|         symbol|           phyticacid|
|     Wilson's disease|        DS|    Wilson's disease|           term|         diseaswilson|
|              Estriol|        CD|             Estriol|           term|              estriol|
|               ATPase|        GP|              ATPase|         symbol|               atpase|
|         breast tumor|        DS|        breast tumor|           term|          breasttumor|
| non-Hodgkin lymphoma|        DS|non-Hodgkin lymphoma|     